In [18]:
import os
import pandas as pd

from langchain.vectorstores import FAISS

from langchain_community.embeddings import HuggingFaceEmbeddings
from smolagents import models, LiteLLMModel, ToolCallingAgent

from src.utils import get_openai_api_key
from src.vector_db_cfg import (
    sports_cfg,
    finance_cfg,
    movie_cfg,
)
from src.retrieval_tool import RetrieverTool

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 1. Connect to LLM

Connect to LLM using OpenAI api.

Create a `llm.env` in root folder:
```
OPENAI_API_KEY='<your-openai-api-key>'
```

In [3]:
# Load openai api key into environment
get_openai_api_key()

In [4]:
LLM_MODEL_ID = "o3-mini"

llm = LiteLLMModel(model_id=LLM_MODEL_ID)

In [9]:
# test llm
test_msg = models.ChatMessage(role="user", content="hello!")

response = llm.generate([test_msg])

print(response.content)
print("model_id:", response.raw.model)

Hello there! How can I help you today?
model_id: o3-mini-2025-01-31


# 2. Create Retrieval Tools

In [14]:
# Setup embedding  model
EMBEDDING_MODEL_NAME = "thenlper/gte-small"

embedding_model = HuggingFaceEmbeddings(
    model_name=EMBEDDING_MODEL_NAME,
    multi_process=True,
    model_kwargs={"device": "cuda"},
    encode_kwargs={"normalize_embeddings": True},
)

In [15]:
data_dir = os.path.join('.', 'data')

In [16]:
# load vector database
db_dir = os.path.join(data_dir, "faiss")

sports_vector_db = FAISS.load_local(
    os.path.join(db_dir, "sports"),
    embeddings=embedding_model,
    allow_dangerous_deserialization=True,
)

finance_vector_db = FAISS.load_local(
    os.path.join(db_dir, "finance"),
    embeddings=embedding_model,
    allow_dangerous_deserialization=True,
)

movie_vector_db = FAISS.load_local(
    os.path.join(db_dir, "movie"),
    embeddings=embedding_model,
    allow_dangerous_deserialization=True,
)

In [19]:
sports_tool = RetrieverTool(vectordb=sports_vector_db, cfg=sports_cfg)
finance_tool = RetrieverTool(vectordb=finance_vector_db, cfg=finance_cfg)
movie_tool = RetrieverTool(vectordb=movie_vector_db, cfg=movie_cfg)

# 3. Create RAG AI Agent

In [43]:
# Using smolagents ToolCallingAgent
agent = ToolCallingAgent(
    model=llm,
    tools=[sports_tool, finance_tool, movie_tool],
    max_steps=10, 
)

#  4. Test Run

In [21]:
question_metadata = pd.read_csv(os.path.join(data_dir, "question_metadata.csv"))
question_metadata.head(2)

,interaction_id,domain,question_type,static_or_dynamic,query,answer
0,47859020-9974-4c81-a897-96594beca8fb,movie,aggregation,static,how many family movies were there that came ou...,109
1,80365e4f-795e-4039-8afb-b7a8e8d54285,movie,post-processing,static,what was the average budget for all movies in ...,"$147,375,000"


In [22]:
# query config
eval_item = question_metadata.loc[14]
question = eval_item['query']
answer = eval_item['answer']

In [24]:
# define query prompt template
enhanced_query = """You are a helpful AI assistant.
You solve tasks step-by-step, using tools when needed.

Available tools are three retriever tools:
1. sports_retriever: contains knowledge about sports, such as basketball, football, etc.
2. finance_retriever: contains knowledge about earnings reports, market trends, stock performance, and economic indicators
3. movie_retriever: contains knowledge about film synopses, box office data, cast information, and critical reception.

Follow this loop:
1. Think about the problem.
2. If a tool is needed, call it with the right input.
3. Observe the result.
4. Repeat until you can give a final answer.

You can only directly answer daily conversation questions, such as "hello", "could you help me find questions?"; when answering factual question, you have to use retriever tools to get answers.
If you can not find relevant information from given retriever tools, just response that you did not find relevant information.


User question: {}
"""

In [25]:
query = enhanced_query.format(question)
print(query)

You are a helpful AI assistant.
You solve tasks step-by-step, using tools when needed.

Available tools are three retriever tools:
1. sports_retriever: contains knowledge about sports, such as basketball, football, etc.
2. finance_retriever: contains knowledge about earnings reports, market trends, stock performance, and economic indicators
3. movie_retriever: contains knowledge about film synopses, box office data, cast information, and critical reception.

Follow this loop:
1. Think about the problem.
2. If a tool is needed, call it with the right input.
3. Observe the result.
4. Repeat until you can give a final answer.

You can only directly answer daily conversation questions, such as "hello", "could you help me find questions?"; when answering factual question, you have to use retriever tools to get answers.
If you can not find relevant information from given retriever tools, just response that you did not find relevant information.


User question: is dreamworks animation owned 

In [44]:
response = agent.run(query)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ You are a helpful AI assistant.                                                                                 │
│ You solve tasks step-by-step, using tools when needed.                                                          │
│                                                                                                                 │
│ Available tools are three retriever tools:                                                                      │
│ 1. sports_retriever: contains knowledge about sports, such as basketball, football, etc.                        │
│ 2. finance_retriever: contains knowledge about earnings reports, market trends, stock performance, and economic │
│ indicators                                                                                                      │
│ 3. movie_retriever: contains knowledge about film synopses, box office data, cast information, and critical     │
│ reception.                                                                                                      │
│                                                                                                                 │
│ Follow this loop:                                                                                               │
│ 1. Think about the problem.                                                                                     │
│ 2. If a tool is needed, call it with the right input.                                                           │
│ 3. Observe the result.                                                                                          │
│ 4. Repeat until you can give a final answer.                                                                    │
│                                                                                                                 │
│ You can only directly answer daily conversation questions, such as "hello", "could you help me find             │
│ questions?"; when answering factual question, you have to use retriever tools to get answers.                   │
│ If you can not find relevant information from given retriever tools, just response that you did not find        │
│ relevant information.                                                                                           │
│                                                                                                                 │
│                                                                                                                 │
│ User question: is dreamworks animation owned by time warner or universal pictures?                              │
│                                                                                                                 │
╰─ LiteLLMModel - o3-mini ────────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'movie_retriever' with arguments: {'query': 'DreamWorks Animation ownership details: Is           │
│ DreamWorks Animation owned by Time Warner or Universal Pictures? Explanation based on movie industry info.'}    │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Observations: Retrieved Documents:
===== Document 0 =====
Template:About Template:Use mdy dates
DreamWorks Pictures (also known as DreamWorks SKG or DreamWorks Studios, commonly referred to as DreamWorks) is an 
American film production label of Amblin Partners. It was formerly distributing its own and third-party films by 
itself. It has produced or distributed more than ten films with box-office grosses of more than $100 million each. 
As of October 2016, DreamWorks' films are marketed and distributed by Universal Pictures.
DreamWorks began in 1994 as an attempt by media moguls Steven Spielberg, Jeffrey Katzenberg and David Geffen 
(together, SKG) to create a new Hollywood studio of which they owned 72%. Currently, DreamWorks operates out of 
offices at Universal Studios.
In December 2005, the founders agreed to sell the studio to Viacom, parent of Paramount Pictures. The sale was 
completed in February 2006.
In 2008, DreamWorks announced its intention to end its partnership with Paramount and signed a $1.2 billion deal to
produce films with India's Reliance ADA Group.|2] Reliance provided $500M of equity to fund recreating DreamWorks 
SKG into DreamWorks Studios, an independent entity. Clark Hallren, former Managing Director of the Entertainment 
Industries group of J. P. Morgan Securities, and Alan J. Levine of J. P. Morgan Entertainment Advisors, led the 
Reliance team in restructuring the company.|3]|4]|5] The following year, DreamWorks entered into a distribution 
agreement with Walt Disney Studios, wherein Disney would distribute DreamWorks films through Touchstone Pictures; 
the deal continued until 2016. After the formation of Amblin Partners in 2015, the studio entered into a 
distribution agreement with Universal Pictures.
DreamWorks' animation arm was spun off in 2004 into DreamWorks Animation SKG (DWA), which currently owns the 
DreamWorks trademarks, and as of August 2016 is a subsidiary of NBCUniversal.|6] Spielberg's company continues to 
use the DreamWorks trademarks under license from Universal Studios.|7]|8]
History|]
DreamWorks SKG founding and Universal Distribution|]===== Document 1 =====
Universal Pictures (also known as Universal Studios) is an American film studio founded in 1912. It was owned by 
NBCUniversal, a division of Comcast.
Universal itself was the film (outside of America) and video distributor for DreamWorks live-action titles and DWA 
titles (Back when DreamWorks Animation was a division of DW before it was spun off in 2004), DreamWorks' theatrical
films from 1998-2006 in American however were distributed by Universal, but through their then DreamWorks Pictures 
label. DreamWorks was then sold to Paramount Pictures in 2006, at which point Paramount took over distribution.
After Viacom spun off DreamWorks Studios in 2008, Universal was planned to resume distributing DreamWorks' movies, 
but this deal fell through as Disney took over the distribution for DreamWorks movies under the Touchstone Pictures
banner starting in 2011 until 2016. In April 2016, Comcast (which owns Universal) purchased DreamWorks Animation 
for $3.8 Billion. It is therefore likely that when the current distribution deal with 20th Century Fox finishes, 
DreamWorks Animation's films will be distributed by Universal. How to Train Your Dragon: The Hidden World (2019) 
was the first DWA film distributed by Universal and as a Universal subsidiary.
Ever since DreamWorks Pictures started, the studio had rented space from Universal Studios on their backlot. This 
continued through the Paramount and Touchstone eras.
Units|]
Divisions|]
- Universal Television
- Universal Pictures Home Entertainment
- Focus Features
- Gramercy Pictures
- NBCUniversal Entertainment Japan
- Working Title Films
- DreamWorks Animation
- Illumination
- Universal Animation Studios
- Universal Cartoon Studios
Dreamworks Filmography|]
- How to Train Your Dragon: The Hidden World (2019)
- Abominable (2019)
- Trolls World Tour (2020)
- The Croods: A New Ag

[Step 1: Duration 19.20 seconds| Input tokens: 1,533 | Output tokens: 246]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'final_answer' with arguments: {'answer': 'DreamWorks Animation is owned by NBCUniversal, which   │
│ is part of Universal Pictures, not Time Warner.'}                                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Observations: DreamWorks Animation is owned by NBCUniversal, which is part of Universal Pictures, not Time Warner.

Final answer: DreamWorks Animation is owned by NBCUniversal, which is part of Universal Pictures, not Time Warner.

[Step 2: Duration 2.97 seconds| Input tokens: 6,572 | Output tokens: 422]

In [42]:
print("- RAG response:", response)
print("- True answer:", answer)

- RAG response: DreamWorks Animation is currently owned by NBCUniversal, the media conglomerate associated with Universal Pictures. Therefore, it is owned by Universal Pictures rather than Time Warner.
- True answer: universal pictures
